In [ ]:
import matplotlib.pyplot as plt
import mygene
import numpy as np
import pandas as pd
import pickle
import random

from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score

np.random.seed(1)
random.seed(1)

In [2]:
import util
import importlib
importlib.reload(util)

N_genes, N_dims, gpt_embeddings = util.retrieve_gpt_gene_embeddings()
random_embeddings = util.generate_random_embeddings(N_genes, N_dims, gpt_embeddings.keys())

### Gene-Gene Interactions

[Du et. al](https://github.com/yiqunchen/GenePT)

In [9]:
GGI_train_text = pd.read_csv(util.data_dir / "gene2vec-predictionData/train_text.txt", sep=" ", header=None)
GGI_train_label = pd.read_csv(util.data_dir / "gene2vec-predictionData/train_label.txt", header=None)

GGI_train_text.columns = ["gene_1","gene_2"]
GGI_train_label.columns = ["label"]
GGI_train_data = pd.concat([GGI_train_text, GGI_train_label], axis=1)
print(GGI_train_data)

         gene_1   gene_2  label
0         GPNMB     BAP1      1
1         GPR34   CARD16      0
2          ELF5    TGFB2      1
3        LILRB2     NCR2      1
4         CRMP1     DLX5      1
...         ...      ...    ...
263011    HOXC5  NEUROG1      1
263012   KCNK18    DEFA3      0
263013  CSRP2BP  SLC5A11      0
263014    SPRY2     ECE2      1
263015    CHRM3   DUSP22      1

[263016 rows x 3 columns]


In [10]:
GGI_test_text = pd.read_csv(util.data_dir / "gene2vec-predictionData/test_text.txt", sep=" ", header=None)
GGI_test_label = pd.read_csv(util.data_dir / "gene2vec-predictionData/test_label.txt", header=None)

GGI_test_text.columns = ["gene_1","gene_2"]
GGI_test_label.columns = ["label"]
GGI_test_data = pd.concat([GGI_test_text, GGI_test_label], axis=1)
print(GGI_test_data)

        gene_1    gene_2  label
0        ALDOB   SIGMAR1      0
1        NRBP2     INTS1      0
2      SLC29A2      NQO2      0
3         GLI1     MAML3      1
4        PDSS1   CSNK1G1      0
...        ...       ...    ...
21443  ADORA2A       PLG      1
21444  PCDHB14    SLC1A1      1
21445   FBXO32  SERPINB9      0
21446     JPH2   OR14A16      0
21447      CNP    KCNMB1      1

[21448 rows x 3 columns]


In [ ]:
def GGI_data_to_embeddings(GGI_data, embeddings, combine_embeddings):
    X, y = [], []
    for _, row in GGI_data.iterrows():
        if row["gene_1"] in embeddings and row["gene_2"] in embeddings:
            X.append(combine_embeddings(embeddings[row["gene_1"]], embeddings[row["gene_2"]]))
            y.append(row["label"])
    return (np.array(X), np.array(y))

combine_embeddings = lambda x, y: (x + y) / 2.0

X_gpt_train, y_gpt_train = GGI_data_to_embeddings(GGI_train_data, gpt_embeddings, combine_embeddings)
X_gpt_test, y_gpt_test = GGI_data_to_embeddings(GGI_test_data, gpt_embeddings, combine_embeddings)

X_random_train, y_random_train = GGI_data_to_embeddings(GGI_train_data, random_embeddings, combine_embeddings)
X_random_test, y_random_test = GGI_data_to_embeddings(GGI_test_data, random_embeddings, combine_embeddings)

In [ ]:
g2v_embeddings_df = pd.read_csv(util.data_dir / "gene2vec-pre_trained_emb/gene2vec_dim_200_iter_9.txt", header=None, sep="\t")
print(gene2vec_embeddings_df)

g2v_embeddings = {}
for _, row in gene2vec_embeddings_df.iterrows():
    embedding = list(map(float, row[1].split()))
    g2v_embeddings[row[0]] = np.array(embedding)

X_g2v_train, y_g2v_train = GGI_data_to_embeddings(GGI_train_data, g2v_embeddings, combine_embeddings)
X_g2v_test, y_g2v_test = GGI_data_to_embeddings(GGI_test_data, g2v_embeddings, combine_embeddings)
print("Train dims: ", X_g2v_train.shape)

In [ ]:
def logistic_regression_eval(X_train, y_train, X_test):
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    lr_pred = lr.predict_proba(X_test)
    return (lr, lr_pred)

lr_gpt, lr_gpt_pred = logistic_regression_eval(X_gpt_train, y_gpt_train, X_gpt_test)
lr_random, lr_random_pred = logistic_regression_eval(X_random_train, y_random_train, X_random_test)
lr_g2v, lr_g2v_pred = logistic_regression_eval(X_g2v_train, y_g2v_train, X_g2v_test)

In [ ]:
fpr_gpt_lr, tpr_gpt_lr, _ = roc_curve(y_gpt_test, lr_gpt_pred[:, 1])
fpr_random_lr, tpr_random_lr, _ = roc_curve(y_random_test, lr_random_pred[:, 1])
fpr_g2v_lr, tpr_g2v_lr, _ = roc_curve(y_g2v_test, lr_g2v_pred[:, 1])

In [ ]:
plt.figure()
plt.figure(figsize=(6,5))

plt.plot(fpr_gpt_lr, tpr_gpt_lr, color='blue', lw=2, 
         label='GenePT (AUC = %0.2f)' % (roc_auc_score(y_gpt_test, lr_gpt_pred[:,1])))
plt.plot(fpr_random_lr, tpr_random_lr, color='red', lw=2, 
         label='Random Embed (AUC = %0.2f)' % (roc_auc_score(y_random_test, lr_random_pred[:,1])))
plt.plot(fpr_g2v_lr, tpr_g2v_lr, color='cyan', lw=2, 
         label='Gene2vec (AUC = %0.2f)' % (roc_auc_score(y_g2v_test, lr_g2v_pred[:,1])))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for gene-gene interaction prediction with a logistic regression classifier')
plt.legend(loc='upper left',bbox_to_anchor=(-0.3, -0.15),fontsize=11,ncol=3)

In [ ]:
def random_forest_eval(X_train, y_train, X_test):
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    rf_pred = rf.predict_proba(X_test)
    return (rf, rf_pred)

rf_gpt, rf_gpt_pred = random_forest_eval(X_gpt_train, y_gpt_train, X_gpt_test)
rf_random, rf_random_pred = random_forest_eval(X_random_train, y_random_train, X_random_test)
rf_g2v, rf_g2v_pred = random_forest_eval(X_g2v_train, y_g2v_train, X_g2v_test)

In [ ]:
fpr_gpt_rf, tpr_gpt_rf, _ = roc_curve(y_gpt_test, rf_gpt_pred[:, 1])
fpr_random_rf, tpr_random_rf, _ = roc_curve(y_random_test, rf_random_pred[:, 1])
fpr_g2v_rf, tpr_g2v_rf, _ = roc_curve(y_g2v_test, rf_g2v_pred[:, 1])

In [ ]:
plt.figure()
plt.figure(figsize=(6,5))

plt.plot(fpr_gpt_rf, tpr_gpt_rf, color='blue', lw=2, 
         label='GenePT (AUC = %0.2f)' % (roc_auc_score(y_gpt_test, rf_gpt_pred[:,1])))
plt.plot(fpr_random_rf, tpr_random_rf, color='red', lw=2, 
         label='Random Embed (AUC = %0.2f)' % (roc_auc_score(y_random_test, rf_random_pred[:,1])))
plt.plot(fpr_g2v_rf, tpr_g2v_rf, color='cyan', lw=2, 
         label='Gene2vec (AUC = %0.2f)' % (roc_auc_score(y_g2v_test, rf_g2v_pred[:,1])))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for gene-gene interaction prediction with a random forest classifier')
plt.legend(loc='upper left',bbox_to_anchor=(-0.3, -0.15),fontsize=11,ncol=3)